In [71]:
import numpy as np
import tensorflow as tf

rng = np.random.default_rng(0)

In [8]:
N = 5

In [132]:
t = 1
U = 1
mu = 1

delta_tau = 1/5
L = 10

lamb = np.arccosh(np.exp((U * delta_tau)/2))

print(f"T: {1/beta}")

T: 0.5


In [35]:
tunneling = np.zeros((N, N))

tunneling += np.diag(np.full(N-1, -t), k=-1)
tunneling += np.diag(np.full(N-1, -t), k=+1)

tunneling[4, 0] += -t
tunneling[0, 4] += -t

tunneling

array([[ 0., -1.,  0.,  0., -1.],
       [-1.,  0., -1.,  0.,  0.],
       [ 0., -1.,  0., -1.,  0.],
       [ 0.,  0., -1.,  0., -1.],
       [-1.,  0.,  0., -1.,  0.]])

In [36]:
kinetic = delta_tau * (tunneling + np.diag(np.full(N, -mu)))
kinetic

array([[-0.2, -0.2,  0. ,  0. , -0.2],
       [-0.2, -0.2, -0.2,  0. ,  0. ],
       [ 0. , -0.2, -0.2, -0.2,  0. ],
       [ 0. ,  0. , -0.2, -0.2, -0.2],
       [-0.2,  0. ,  0. , -0.2, -0.2]])

In [59]:
field = rng.choice([-1, +1], size=(N, L))

field

array([[ 1,  1,  1, -1, -1, -1, -1, -1, -1,  1],
       [ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1],
       [-1,  1,  1, -1, -1,  1,  1, -1,  1,  1],
       [ 1, -1, -1,  1, -1,  1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1,  1,  1,  1, -1,  1]])

In [133]:
v_up = lamb * tf.cast(tf.linalg.diag(field.T), dtype=tf.float64)
v_down = -v_up

v = tf.constant(np.array([v_up, v_down]))
v

<tf.Tensor: shape=(2, 10, 5, 5), dtype=float64, numpy=
array([[[[ 0.45470309,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.45470309,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        , -0.45470309,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.45470309,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
          -0.45470309]],

        [[ 0.45470309,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.45470309,  0.        ,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.45470309,  0.        ,
           0.        ],
         [ 0.        ,  0.        ,  0.        , -0.45470309,
           0.        ],
         [ 0.        ,  0.        ,  0.        ,  0.        ,
          -0.45470309]],

        [[ 0.45470309,  0.        ,  0.        ,  0.        ,
           0.      

In [129]:
G = np.zeros((2, N, N))

for spin in range(2):
    product = tf.constant(np.eye(N))
    for l in range(L-1, 0, -1):
        product = tf.linalg.expm(kinetic) @ tf.linalg.expm(v[spin][l]) @ product
    G[spin] = tf.linalg.inv(np.eye(N) + product)

G = tf.constant(G)

G

<tf.Tensor: shape=(2, 5, 5), dtype=float64, numpy=
array([[[ 7.06713731e-01,  3.50478008e-01, -1.22049570e-05,
         -7.74308846e-02,  4.16330248e-01],
        [ 2.16461550e-01,  3.04341570e-01,  2.52797400e-01,
         -5.73694700e-03, -5.51334187e-02],
        [ 2.22011740e-02,  2.94402791e-01,  4.66803076e-01,
          1.14336165e-01, -2.22648273e-01],
        [-8.80482922e-02, -4.54237983e-02,  2.37543157e-01,
          9.16929771e-01,  2.40063490e-01],
        [ 1.36008522e-01, -5.26202878e-02, -8.60074087e-02,
          6.36240985e-02,  7.20263853e-01]],

       [[ 5.66003621e-01,  1.41049128e-01, -4.59032264e-02,
          1.46274114e-04,  1.81660674e-01],
        [ 2.41621855e-01,  9.03223271e-01,  6.55214772e-02,
         -1.37713738e-01, -7.60577278e-02],
        [-1.45795130e-01,  8.26977059e-02,  8.96104339e-01,
          3.21549455e-01,  1.69290320e-02],
        [ 1.99634562e-03, -4.12061114e-02,  1.29451782e-01,
          2.50263362e-01,  1.26045026e-01],
        [ 6

In [143]:
i = 0
l = L-1

d_up   = 1 + (1 - G[0][0, 0])*(np.exp(-2*lamb*field[i, l]) - 1)
d_down = 1 + (1 - G[1][0, 0])*(np.exp(+2*lamb*field[i, l]) - 1)
d = d_up * d_down

d

<tf.Tensor: shape=(), dtype=float64, numpy=1.3556640606698303>

In [142]:
r = rng.random()

print(field[i, l])
if r < d: field[i, l] *= -1
print(field[i, l])

1


NameError: name 'd' is not defined